In [18]:
from pyswip.prolog import Prolog
from pyswip import registerForeign, Functor, call, Variable
import tempfile
import os
import sys

## Interface Scaffold

In [22]:
debug = False
prolog = Prolog()
prolog.consult("./buenos_aires-kb.pl")

# pull out Prolog built-ins into the Python name-space
retractall = Functor("retractall")
known = Functor("known", 3)
member = Functor('member', 2)

question_dict = {
    'budget': "Should the activity be free? (yes/no)",
    'experience': "Which of the following experiences would you like to have? (Input the associated number)",
    'time_required': "How long should the activity take? (Short is less than 2 hours)",
    'distance': "Do you want to do something nearby or far away (Input the associated number)",
    'price_point': "Roughly how much do you want to pay for your food? (Input the associated number",
    'culture_type': "What type of cultural experience would you like to have? (Input the associated number)",
    'museum_type': "What type of museum would you like to visit? (Input the associated number)",
    'diet': "Are you vegetarian (yes/no)"
}

# user input and print functions
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True

def read_py(A,V,Y):
    if isinstance(Y, Variable):
        response = input(question_dict[str(A)])
        processed_response = response.lower().strip()
        Y.unify(processed_response)
        return True
    else:
        return False

def read_menu_py(A, V, X, Menu):
    if isinstance(X, Variable):
        menu_dict = {str(i+1): str(Menu[i]) for i in range(len(Menu))}
        options = '\n'.join([f'{i}. {j}' for i, j in menu_dict.items()])
        response = input(f"{question_dict[str(A)]}: {options}")
        # fail gracefully on incorrect input and try again
        X.unify(menu_dict.get(response, str(response)))
        return True
    else:
        return False
    
def ask_menu_again_py(Z, A, V):
    print(f"{Z} is not a legal value, try again.")
    return True

write_py.arity = 1
read_py.arity = 3
ask_menu_again_py.arity=3
read_menu_py.arity = 4

registerForeign(read_py)
registerForeign(read_menu_py)
registerForeign(write_py)
registerForeign(ask_menu_again_py)

call(retractall(known))
destination = [s for s in prolog.query("findall(Location, recommended(Location), List).")]
locations = list(set(destination[0]['List']))

if debug:
    known_attrs = [s for s in prolog.query("findall(X, known(yes, X, _), List).")]
    known_vals = [s for s in prolog.query("findall(Y, known(yes, _, Y), List).")]
    print("\n KNOWN ATTRS:", known_attrs)
    print("\n KNOWN VALS:", known_vals)

if not locations:
    print("unknown")
elif len(locations) == 1:
    print(f"Your recommended destination is {locations[0]}")
else:
    print(f"Some recommended destinations are {', '.join(locations[:-1])} and {locations[-1]}")

 is not a legal value, try again.
 is not a legal value, try again.
 is not a legal value, try again.
 is not a legal value, try again.
 is not a legal value, try again.


Exception ignored on calling ctypes callback function: <function _foreignWrapper.<locals>.wrapper at 0x1079fed30>
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/pyswip/easy.py", line 536, in wrapper
    r = fun(*args)
  File "/var/folders/_n/c17wd9k97s7dm583pk_qqyjh0000gn/T/ipykernel_85730/1269884622.py", line 40, in read_menu_py
  File "/opt/homebrew/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 1075, in raw_input
    return self._input_request(
  File "/opt/homebrew/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 1120, in _input_request
    raise KeyboardInterrupt("Interrupted by user") from None
KeyboardInterrupt: Interrupted by user


PrologError: Caused by: 'findall(Location, recommended(Location), List).'. Returned: 'error(domain_error(foreign_return_value, 105553136083296), context(/(read_menu_py, 4), _250))'.

## Testing Area

In [4]:
prolog = Prolog()

# pull out Prolog built-ins into the Python name-space
retractall = Functor("retractall")
known = Functor("known", 3)
member = Functor('member', 2)

# user input and print functions
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True

def read_py(A,V,Y):
    if isinstance(Y, Variable):
        response = input(str(A) + " is " + str(V) + "? ")
        Y.unify(response)
        return True
    else:
        return False

def read_menu_py(A, V, X, Menu):
    if isinstance(X, Variable):
        response = input(f"What is the value for {str(A)}? Options: {[str(i) for i in Menu]}")
        X.unify(response)
        return True
    else:
        return False
    
def ask_menu_again(Z, A, V):
    print(f"{Z} is not a legal value, try again.")
    return True

write_py.arity = 1
read_py.arity = 3
ask_menu_again.arity=3
read_menu_py.arity = 4

registerForeign(read_py)
registerForeign(read_menu_py)
registerForeign(write_py)
registerForeign(ask_menu_again)

# Consult the knowledge base 
prolog.consult("./test-kb.pl")

call(retractall(known))
# destination = [s for s in prolog.query("destination(X).", maxresult=1)]
# print("Your recommended destination is " + (destination[0]['X'] + "." if destination else "unknown."))

# destination = [s for s in prolog.query("destination(X).", maxresult=10)]
# print("Your recommended destination is " + (destination[0]['X'] + "." if destination else "unknown."))
# print(destination)|
destination = [s for s in prolog.query("findall(Location, destination(Location), List).")]
locations = list(set(destination[0]['List']))
print("DESTINATION", destination)
if not locations:
    print("unknown")
elif len(locations) == 1:
    print(f"Your recommended destination is {locations[0]}")
else:
    print(f"Some recommended destinations are {', '.join(locations[:-1])} and {locations[-1]}")

medium

DESTINATION [{'Location': Variable(78), 'List': []}]
unknown


In [92]:
KB = """
distance(museum, 5).
distance(monument, 3).
distance(fairground, 3).

distance_appropriate(Location) :- distance(Location, X), X =< 4, distance(close).
distance_appropriate(Location) :- distance(Location, X), X > 4, \+ distance(close).

destination(X) :- distance_appropriate(X).

distance(X) :- ask(distance, X).

ask(A, V):-
known(yes, A, V), % succeed if true
!.	% stop looking

ask(A, V):-
known(_, A, V), % fail if false
!, fail.

% If not multivalued, and already known to be something else, don't ask again for a different value.
ask(A, V):-
\+multivalued(A),
known(yes, A, V2),
V \== V2,
!, fail.

ask(A, V):-
read_py(A,V,Y), % get the answer
assertz(known(Y, A, V)), % remember it
atom_string(Z, Y),
Z == yes.	% succeed or fail

"""

In [ ]:
prolog = Prolog()

# pull out Prolog built-ins into the Python name-space
retractall = Functor("retractall")
known = Functor("known", 3)
member = Functor('member', 2)

# user input and print functions
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True

def read_py(A,V,Y):
    if isinstance(Y, Variable):
        response = input(str(A) + " is " + str(V) + "? ")
        Y.unify(response)
        return True
    else:
        return False

def read_menu_py(A, V, X, Menu):
    if isinstance(X, Variable):
        response = input(f"What is the value for {str(A)}? Options: {[str(i) for i in Menu]}")
        X.unify(response)
        return True
    else:
        return False
    
def ask_menu_again(Z, A, V):
    print(f"{Z} is not a legal value, try again.")
    return True

write_py.arity = 1
read_py.arity = 3
ask_menu_again.arity=3
read_menu_py.arity = 4

registerForeign(read_py)
registerForeign(read_menu_py)
registerForeign(write_py)
registerForeign(ask_menu_again)

# Create a temporary file with the KB in it
(FD, name) = tempfile.mkstemp(suffix='.pl', text = "True")
with os.fdopen(FD, "w") as text_file:
    text_file.write(KB)
prolog.consult(name) # open the KB for consulting
os.unlink(name) # Remove the temporary file

call(retractall(known))
# destination = [s for s in prolog.query("destination(X).", maxresult=1)]
# print("Your recommended destination is " + (destination[0]['X'] + "." if destination else "unknown."))

destination = [s for s in prolog.query("findall(Location, destination(Location), List).")]
locations = list(set(destination[0]['List']))
print("DESTINATION", destination)
if not locations:
    print("unknown")
elif len(locations) == 1:
    print(f"Your recommended destination is {locations[0]}")
else:
    print(f"Some recommended destinations are {', '.join(locations[:-1])} and {locations[-1]}")

## Old Rules & Askables

In [7]:
# """
# % Rule 1:** if engine is not turning_over and battery is bad then problem is battery.

# destination(battery):- \+engine(turning_over), battery(bad).

# % Rule 2

# problem(engine_oil_low) :- \+engine(turning_over), warning_light(oil).

# % Rule 3

# battery(bad) :- lights(weak).

# % Rule 4

# battery(bad) :- radio(weak).

# % Rule 5

# problem(engine_flooded):- engine(turning_over), smell(gas).

# % Rule 6 if engine is turning_over and gas_gauge is empty then problem is out_of_gas.

# problem(out_of_gas):- engine(turning_over), gas_gauge(empty).


# % Askables

# engine(X):- ask(engine, X).

# warning_light(X):- ask(warning_light, X).

# lights(X):- ask(lights, X).

# smell(X):- ask(smell, X).

# radio(X):- ask(radio, X).

# gas_gauge(X):- ask(gas_gauge, X).
# """

# Template Example

In [2]:
KB = """
%  Tell prolog that known/3 and multivalued/1 will be added later
:- dynamic known/3, multivalued/1.

% Enter your KB below this line:

problem(battery) :- \+engine(turning_over), battery(bad).
problem(engine_oil_low) :- \+engine(turning_over), warning_light(oil).
battery(bad) :- lights(weak).
battery(bad) :- radio(weak).
problem(out_of_gas) :- engine(turning_over), gas_gauge(empty).
problem(engine_flooded) :- engine(turning_over), smell(gas).
problem(dirty_car) :- smell(pizza).

% The code below implements the prompting to ask the user:

gas_gauge(X) :- ask(gas_gauge, X).
engine(X) :- ask(engine, X).
lights(X) :- ask(lights, X).
radio(X) :- ask(radio, X).
smell(X) :- ask(smell, X).
warning_light(X) :- ask(warning_light,X).

% Asking clauses

ask(A, V):-
known(yes, A, V), % succeed if true
!.	% stop looking

ask(A, V):-
known(_, A, V), % fail if false
!, fail.

% If not multivalued, and already known to be something else, don't ask again for a different value.
ask(A, V):-
\+multivalued(A),
known(yes, A, V2),
V \== V2,
!, fail.

ask(A, V):-
read_py(A,V,Y), % get the answer
assertz(known(Y, A, V)), % remember it
Y == "yes".	% succeed or fail
"""

In [1]:
# Import necessary packages
import tempfile
from pyswip.prolog import Prolog
from pyswip.easy import *

prolog = Prolog() # Global handle to interpreter

retractall = Functor("retractall")
known = Functor("known",3)

# Define foreign functions for getting user input and writing to the screen
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True

def read_py(A,V,Y):
    if isinstance(Y, Variable):
        response = input(str(A) + " is " + str(V) + "? ")
        Y.unify(response)
        return True
    else:
        return False

write_py.arity = 1
read_py.arity = 3
registerForeign(read_py)
registerForeign(write_py)

prolog.consult('knowledge-base.pl') # open the KB for consulting

call(retractall(known))
problem = [s for s in prolog.query("problem(X).", maxresult=1)]
print("Your problem is " + (problem[0]['X'] + "." if problem else "unknown."))

Your problem is battery.
